<img src="images/econ140R_logo.png" width="200" />

<h1>ECON 140R Class 11</h1>

<h2>IPUMS</h2>
In this hands-on, we will step through an example of loading up and recoding a data extract from the expansive Integrated Public Use Microdata Series ([IPUMS](https://www.ipums.org)) database.

<h2>Learning objectives:</h2>

1. More experience with OLS
2. Better formatting, with `options(scipen=999)` and `table()` inside a `data.frame`
3. Recoding variables using `ifelse()` and `case_when()` inside the `dplyr` library
4. Estimation with survey weights

In [21]:
library(ipumsr)
library(dplyr)
library(tidyverse)
library(haven)

It turns out that this handy command stops __R__ from defaulting to scientific notation.

In [22]:
options(scipen=999)

<h2>Using IPUMS</h2>

IPUMS data must be downloaded from their website (or analyzed online, but I don't recommend that option) using an established user account. Like with almost everything in __R__, there are at least 3 ways of working with IPUMS data once you've downloaded your extract from IPUMS and either uploaded it to datahub or installed your own (free) local copy of [RStudio desktop](https://www.rstudio.com/products/rstudio/) to examine the data.

There are at least three methods:
1. Download .DAT and .XML files from IPUMS and use `ipumsr`
2. Download Stata .DTA files from IPUMS and use `read_dta()` from `haven`
3. Download .CSV files from IPUMS and use `read.csv()` in base __R__

In this notebook, we'll look at the first way: using the `ipumsr` library functions below to read in the data from a .DAT and .XML (a.k.a. DDI) file. To pull this off, you need to have the .DAT and .XML files downloaded and placed into your working directory.

The following commands pull in my Current Population Survey (CPS) extract `cps_00028`. The file name has some meaning because it will start with the part of IPUMS you've accessed. But the numeric part is unique to you as the user; it just indexes how many queries you've made.

The extract drew these variables from the January 2018 wave (job tenure supplement) of the Current Population Survey on IPUMS:

````{verbatim}
YEAR            Survey year
SERIAL          Household serial number
MONTH           Month
HWTFINL         Household weight, Basic Monthly
CPSID           CPSID, household record
STATEFIP        State (FIPS code)
METRO           Metropolitan central city status
PERNUM          Person number in sample unit
WTFINL          Final Basic Weight
CPSIDP          CPSID, person record
AGE             Age
SEX             Sex
RACE            Race
MARST           Marital status
VETSTAT         Veteran status
HISPAN          Hispanic origin
EMPSTAT         Employment status
OCC             Occupation
IND             Industry
CLASSWKR        Class of worker
AHRSWORKT       Hours worked last week
WKSTAT          Full or part time status
EDUC            Educational attainment recode
EDCYC           Years of college credit completed
EDDIPGED        High school or GED
EDHGCGED        Highest grade completed before receiving GED
JTYEARS         Length of time worked at current job in years
JTYEARAGO       Employment one year ago
JTSAME          Doing the same work a year ago
JTYPE           Type of business or organization in which you worked last year
JTCLASS         Class of worker, Job Tenure Supplement
JTIND           Industry one year ago
JTOCC           Occupation one year ago
JTSUPPWT        Job tenure supplement weight
EARNWEEK        Weekly earnings
````

These files are LOCAL, stored on datahub. The IPUMS model typically requires you to download the dataset and the XML file and place them somewhere. Here, I have downloaded them from IPUMS to my machine, and then I uploaded them to datahub inside the `data` folder in our working directory for class hands-on, `Class Meetings`.

In [23]:
cps_data <- ipumsr::read_ipums_micro(
  ddi = "data/cps_00028.xml",
  data = "data/cps_00028.dat.gz"
)

Use of data from IPUMS CPS is subject to conditions including that users should
cite the data appropriately. Use command `ipums_conditions()` for more details.


In [24]:
# Here is another version of calls that successfully load the data, 
# here drawing from the unzipped data file cps_00028.dat instead of the gzipped compressed file

#cps_ddi <- read_ipums_ddi("data/cps_00028.xml")
#cps_data <- read_ipums_micro(cps_ddi, verbose = FALSE)

In [25]:
head(cps_data)

YEAR,SERIAL,MONTH,HWTFINL,CPSID,STATEFIP,METRO,PERNUM,WTFINL,CPSIDP,⋯,EDHGCGED,JTYEARS,JTYEARAGO,JTSAME,JTYPE,JTCLASS,JTIND,JTOCC,JTSUPPWT,EARNWEEK
<dbl>,<dbl>,<int+lbl>,<dbl>,<dbl>,<int+lbl>,<int+lbl>,<dbl>,<dbl>,<dbl>,⋯,<int+lbl>,<dbl+lbl>,<int+lbl>,<int+lbl>,<int+lbl>,<int+lbl>,<dbl>,<dbl>,<dbl>,<dbl>
2018,1,1,1490.589,20161000000100,1,2,1,2158.949,20161000000101,⋯,99,99.99,99,99,99,99,0,0,0.000,9999.99
2018,1,1,1490.589,20161000000100,1,2,2,1490.589,20161000000102,⋯,99,99.99,99,99,99,99,0,0,0.000,9999.99
2018,2,1,1609.490,20161200000200,1,2,1,1420.746,20161200000201,⋯,99,1.75,99,2,99,99,0,0,1652.859,9999.99
2018,2,1,1609.490,20161200000200,1,2,2,1609.490,20161200000202,⋯,99,99.99,99,99,99,99,0,0,0.000,9999.99
2018,3,1,1797.041,20180100000300,1,2,1,2053.275,20180100000301,⋯,99,1.08,99,2,99,99,0,0,2388.727,9999.99
2018,3,1,1797.041,20180100000300,1,2,2,1797.041,20180100000302,⋯,99,5.00,99,2,99,99,0,0,2082.874,9999.99


Let's clean up the data. But what variables need cleaning? You could browse the documentation for each one, or you could look at a quick `summary()` for suspect variables. Seasoned users of IPUMS will know that earnings and income contain real data <i>and codes for missing data</i> as explained for `EARNWEEK` [here for example](https://cps.ipums.org/cps-action/variables/EARNWEEK#codes_section).

In [26]:
summary(cps_data$EARNWEEK)

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
    0.23  9999.99  9999.99  9025.52  9999.99  9999.99 

Let's recode to drop those missing to NA's, creating a new variable `earnweek`. __R__ is case-sensitive, so the main challenge is remembering what we have done ourselves! 

In [27]:
cps_data <- mutate(cps_data, 
                   earnweek = ifelse(EARNWEEK < 9999.99, EARNWEEK, NA)
                  )

Let's check our work:

In [30]:
summary(cps_data$earnweek)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   0.23  460.00  769.23  957.69 1250.00 2884.61  111951 

Looks much better.

And let's create a new variable equal to the log of weekly earnings. Economists pretty much always take the natural log of money variables like earnings, income, or wealth, especially when they are going to appear on the left-hand side of the regression. 

In [31]:
cps_data <- mutate(cps_data, 
                   logearnweek = log(earnweek)
                  )

A standard trick: when gender identity is binary in the data, it's usually coded for male == 1 and female == 2. Simply subtract 1 to get an indicator for female identity. Many Census and historical surveys still call gender identity "sex," while follow a more modern naming approach.

In [32]:
cps_data <- mutate(cps_data, 
                   female = SEX - 1
                  )

The job tenure variable also has NA's coded as numbers, so let's drop those. Here's the check:

In [33]:
summary(cps_data$JTYEARS)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   0.00    9.00   99.99   64.69   99.99   99.99 

In [34]:
cps_data <- mutate(cps_data, 
                   jtyears = ifelse(JTYEARS < 99.98, JTYEARS, NA)
                  )

In [35]:
summary(cps_data$jtyears)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   0.00    1.67    5.00    8.21   12.00   32.00   77210 

Looks reasonable.

The `HISPAN` variable takes on many values, [as shown here in the documentation](https://cps.ipums.org/cps-action/variables/HISPAN#codes_section). None missing, so a useful way forward is to assign 0 for `HISPAN` == 0 and 1 for everything else, which code different Hispanic identities. (Presumably Brazilians in the dataset are coded as `HISPAN` == 612 for "South American," but that's an open question.) This code does this:

In [37]:
cps_data <- mutate(cps_data, 
                   hispanic = ifelse(HISPAN == 0, 0, 1)
                  )

Here is my preferred quadrichotomous measure of race/ethnicity for the U.S. in 2022: 
1. Non-Hispanic white
2. Non-Hispanic Black
3. Hispanic, any race
4. Non-Hispanic other race 

Let's create `raceth` that measures that:

In [38]:
cps_data <- mutate(cps_data, 
                   raceth = ifelse(RACE == 100 & hispanic == 0, 1, 
                                   ifelse(RACE == 200 & hispanic == 0, 2,
                                          ifelse(hispanic == 1, 3, 4)))
                   )

Marital status is measured in `MARST`, and codes 1 and 2 mean the respondent is married, spouse present or absent.

In [39]:
cps_data <- mutate(cps_data, 
                   married = ifelse(MARST == 1 | MARST == 2, 1, 0)
                  )

Let's look at the variable `EDUC`, which codes educational attainment. This code creates a more legible table of tabulated counts per value:

In [40]:
temp = data.frame(table(cps_data$EDUC))
temp

Var1,Freq
<fct>,<int>
1,23144
2,312
10,512
20,1087
30,1994
40,2686
50,3061
60,3501
71,1731


These definitely aren't years of education. As [shown in the documentation](https://cps.ipums.org/cps-action/variables/EDUC#codes_section), this is what we're looking at:

````{verbatim}
*                           NIU or blank |    1
*                      None or preschool |    2
*                   Grades 1, 2, 3, or 4 |   10
*                          Grades 5 or 6 |   20
*                          Grades 7 or 8 |   30
*                                Grade 9 |   40
*                               Grade 10 |   50
*                               Grade 11 |   60
*                 12th grade, no diploma |   71
*      High school diploma or equivalent |   73
*             Some college but no degree |   81
*Associate's degree, occupational/vocati |   91
*   Associate's degree, academic program |   92
*                      Bachelor's degree |  111
*                        Master's degree |  123
*             Professional school degree |  124
*                       Doctorate degree |  125
````

This is an elegant way of recoding `EDUC` into something that economists think is useful: years of education attained. Here, we use `case_when()`, which is part of the `dpylr` package. Special thanks to UC Berkeley Demography Ph.D. student Casey Breen for this code.

The question of exactly how to map attainment categories to years of attainment is a complicated one. I've tried to adapt what [David Jaeger (2003) suggests](https://www.sciencedirect.com/science/article/pii/S0165176502002598).

(Another way of proceeding altogether would be to generate indicator variables for levels of attainment.)

In [41]:
cps_data <- cps_data %>%
mutate(edyrs = case_when(
    EDUC == 2 ~ 0,
    EDUC == 10 ~ 2.5,
    EDUC == 20 ~ 5.5,
    EDUC == 30 ~ 7.5,
    EDUC == 40 ~ 9,
    EDUC == 50 ~ 10,
    EDUC == 60 ~ 11,
    EDUC == 71 ~ 12,
    EDUC == 73 ~ 12,
    EDUC == 81 ~ 14,
    EDUC == 91 ~ 14,
    EDUC == 92 ~ 14,
    EDUC == 111 ~ 16,
    EDUC == 123 ~ 18,
    EDUC == 124 ~ 18,
    EDUC == 125 ~ 18
    ))

Now let's look at what's here:

In [42]:
temp = data.frame(table(cps_data$edyrs))
temp

Var1,Freq
<fct>,<int>
0,312
2.5,512
5.5,1087
7.5,1994
9,2686
10,3061
11,3501
12,30090
14,27680


Here is another bunch of code that is far uglier but produces the same thing, using a ton of nested `ifelse()` calls.

In [43]:
#cps_data <- mutate(cps_data, edyrs = ifelse(EDUC == 2, 0,
#                                            ifelse(EDUC == 10, 2.5,
#                                                  ifelse(EDUC == 20, 5.5,
#                                                        ifelse(EDUC == 30, 7.5,
#                                                              ifelse(EDUC == 40, 9,
#                                                                    ifelse(EDUC == 50, 10,
#                                                                          ifelse(EDUC == 60, 11,
#                                                                                ifelse(EDUC == 71, 12,
#                                                                                      ifelse(EDUC == 73, 12,
#                                                                                            ifelse(EDUC == 81, 14,
#                                                                                                  ifelse(EDUC == 91, 14,
#                                                                                                        ifelse(EDUC == 92, 14,
#                                                                                                              ifelse(EDUC == 111, 16,
#                                                                                                                    ifelse(EDUC == 123, 18,
#                                                                                                                          ifelse(EDUC == 124, 18,
#                                                                                                                                ifelse(EDUC == 125, 18, NA)
#                                                                                                                                ))))))))))))))
#                                            )
#                   )

Finally, here are a few more recodings. Recall our earlier discussion that labor economists like to measure <i>experience</i> as years of age minus years of education. Luckily `AGE` is not goofily coded. It's just years of age:

In [47]:
summary(cps_data$AGE)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   0.00   20.00   40.00   40.03   59.00   85.00 

In [48]:
cps_data <- mutate(cps_data, 
                   exper = AGE - edyrs)

In [49]:
cps_data <- mutate(cps_data, 
                   expersq = exper^2)

With all of these recodings, we are finally ready to go. Let's run this model of log earnings, where the treatment variables of interest are a binary measure of female gender identity and years of education, and we have a set of controls that includes race/ethnicity, years of experience, years of experience squared, and also years of job tenure:

$$
\ln Y_i = \alpha + \beta^f \ female_i + \beta^e \ educ_i + B \ controls_i + \epsilon_i
$$

Let's also restrict the sample to workers aged 25-64. We do that with a call to `subset()` in the `data = ` option.

In [51]:
cps_reg1 <- lm(logearnweek ~ female 
               + edyrs 
               + factor(raceth) 
               + exper + expersq 
               + jtyears, 
               data = subset(cps_data, AGE >= 25 & AGE <= 64)
               )
summary(cps_reg1)


Call:
lm(formula = logearnweek ~ female + edyrs + factor(raceth) + 
    exper + expersq + jtyears, data = subset(cps_data, AGE >= 
    25 & AGE <= 64))

Residuals:
    Min      1Q  Median      3Q     Max 
-8.2460 -0.3124  0.0553  0.4172  2.6295 

Coefficients:
                   Estimate  Std. Error t value             Pr(>|t|)    
(Intercept)      4.91807082  0.06655212  73.898 < 0.0000000000000002 ***
female          -0.36884313  0.01438936 -25.633 < 0.0000000000000002 ***
edyrs            0.10686433  0.00308434  34.647 < 0.0000000000000002 ***
factor(raceth)2 -0.14380309  0.02531356  -5.681  0.00000001381863398 ***
factor(raceth)3 -0.06483196  0.02308744  -2.808              0.00499 ** 
factor(raceth)4 -0.08274355  0.02620348  -3.158              0.00160 ** 
exper            0.02694388  0.00342620   7.864  0.00000000000000415 ***
expersq         -0.00045188  0.00005626  -8.032  0.00000000000000108 ***
jtyears          0.01684017  0.00098428  17.109 < 0.0000000000000002 ***
---
Sign

Do you remember the tricks about quadratics? What is the $x$-coordinate of the vertex? If the (partial) function we have is:

$$
\ln Y_i = a \cdot exper_i^2 + b \cdot exper_i + c
$$

then the vertex falls where $exper_i = -b \ / \ 2a$. You can also find this by differentiating the formula above and setting the derivative to zero, which reveals that $2a \cdot exper_i + b = 0$, and solving for $exper_i$. 

Use this formula and plug in the coefficient on `exper` for $b$ and the coefficient on `expersq` for $a$ to find:

In [56]:
0.02694388/(2*0.00045188)

-coef(cps_reg1)["exper"] / (2 * coef(cps_reg1)["expersq"])

[1] 29.81309

exper 
29.81333

In words, log earnings rise with experience up to about year 30. For a college graduate, that would be around age 52.

<hr>

What about marital status? We didn't include that, nor did we control for a bunch of other interesting variables in the data, like veteran status and other characteristics of job tenure. Let's try inserting `married` to see what we find:

In [57]:
cps_reg2 <- lm(logearnweek ~ female 
               + edyrs 
               + factor(raceth) 
               + exper + expersq 
               + jtyears 
               + married, 
               data = subset(cps_data, AGE >= 25 & AGE <= 64)
               )
summary(cps_reg2)


Call:
lm(formula = logearnweek ~ female + edyrs + factor(raceth) + 
    exper + expersq + jtyears + married, data = subset(cps_data, 
    AGE >= 25 & AGE <= 64))

Residuals:
    Min      1Q  Median      3Q     Max 
-8.2322 -0.3099  0.0581  0.4164  2.5780 

Coefficients:
                   Estimate  Std. Error t value             Pr(>|t|)    
(Intercept)      4.93364026  0.06655965  74.124 < 0.0000000000000002 ***
female          -0.36575465  0.01438811 -25.421 < 0.0000000000000002 ***
edyrs            0.10560418  0.00309266  34.147 < 0.0000000000000002 ***
factor(raceth)2 -0.12734148  0.02553041  -4.988    0.000000622033599 ***
factor(raceth)3 -0.06520926  0.02306104  -2.828              0.00470 ** 
factor(raceth)4 -0.08390525  0.02617454  -3.206              0.00135 ** 
exper            0.02430617  0.00346882   7.007    0.000000000002610 ***
expersq         -0.00041620  0.00005671  -7.339    0.000000000000234 ***
jtyears          0.01651850  0.00098558  16.760 < 0.0000000000000002 **

Describe what you see. Are these results fundamentally different or mostly the same as the results without controlling for `married()`?

<hr>

<h2>Sampling weights</h2>

Early samples like the early public-use microdata drawn from the decennial Census were simple 5% cuts of the entire Census. That would mean that out of the [334 million resident population in the U.S. on January 1, 2023](https://www.census.gov/library/stories/2022/12/happy-new-year-2023.html), a straight 5% sample would select 5 out of every 100 people at random, for a total of 16.7 million people. This kind of design is found in the public-use version of the 1980 Census in IPUMS, for example.

But modern surveys usually have a more complex design, because one would like to have statistical power to understand key subgroups. The share of people in the U.S. who identify as Black or African-American was [13.6% in 2022](https://www.census.gov/quickfacts/fact/table/US/PST045222). Not all population subgroup is <i>oversampled</i>, meaning included at higher rates than non-Hispanic white people, but many are. This is done in order to facilitate analysis of well-being among those subgroups. 

<b>Sampling weights</b> are supplied in modern surveys like the CPS and others to help create sample moments, like the sample average, that better approximate population moments. We can use sampling weights in regression analysis using the `weight =` option in `lm()` if we want.

But <b>sampling weights might not change the results of OLS very much</b>, if the regression already controls for demographic variables like age, gender, and race/ethnicity. The reason is that the sampling weights are usually functions of those demographic variables, so estimating OLS with sample weights might be redundant.

We have 2 sampling weights in the extract: `WTFINL` the final basic weight, and `JTSUPPWT` the job tenure supplement weight. If we use job tenure variables in a model, we should use the latter weight. Let's give it a try.

In [59]:
cps_reg3 <- lm(logearnweek ~ female 
               + edyrs 
               + factor(raceth) 
               + exper + expersq 
               + jtyears 
               + married, 
               data = subset(cps_data, AGE >= 25 & AGE <= 64),
               weight = JTSUPPWT
              )
summary(cps_reg3)


Call:
lm(formula = logearnweek ~ female + edyrs + factor(raceth) + 
    exper + expersq + jtyears + married, data = subset(cps_data, 
    AGE >= 25 & AGE <= 64), weights = JTSUPPWT)

Weighted Residuals:
    Min      1Q  Median      3Q     Max 
-547.26  -15.19    2.22   19.90  230.42 

Coefficients:
                  Estimate Std. Error t value             Pr(>|t|)    
(Intercept)      4.9097717  0.0653313  75.152 < 0.0000000000000002 ***
female          -0.3549084  0.0145265 -24.432 < 0.0000000000000002 ***
edyrs            0.1044075  0.0030633  34.084 < 0.0000000000000002 ***
factor(raceth)2 -0.1453486  0.0235690  -6.167   0.0000000007263097 ***
factor(raceth)3 -0.0787602  0.0209908  -3.752             0.000176 ***
factor(raceth)4 -0.1009401  0.0253102  -3.988   0.0000671281216371 ***
exper            0.0264289  0.0034196   7.729   0.0000000000000120 ***
expersq         -0.0004336  0.0000565  -7.674   0.0000000000000184 ***
jtyears          0.0163590  0.0010288  15.901 < 0.0000000000

Describe what you see. Are these results fundamentally different or mostly the same as the results without sampling weights?

<div style="text-align: right"> <span style="font-family:Papyrus; ">And they lived happily ever after. The End.</span></div>